In [3]:
#import modules
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pylab as plt
import seaborn as sns
import credentials
import datetime
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
#access database
URI = "dublinbikeappdb.cxaxe40vwlui.us-east-1.rds.amazonaws.com"
DB = "dbikes1"
name = credentials.name
pw = credentials.password

"""Initate connection"""
engine = create_engine("mysql+mysqlconnector://{}:{}@{}:3306/{}".format(name,pw,URI,DB),echo=False)

In [5]:
#get infor from db (testing with station number 4)
n = 4
availability = '''SELECT *
FROM availability
WHERE number = {}
'''.format(n)

weather = '''SELECT *
FROM weather
'''

select = """SELECT s.number
FROM stations s"""

In [6]:
#convert db tables to pandas dataframes
dfs = pd.read_sql_query(select, engine)
dfa = pd.read_sql_query(availability, engine)
dfw = pd.read_sql_query(weather, engine)

In [7]:
#merge weather and occupancy tables together
df = pd.merge_asof(dfa, dfw, left_on="last_update", right_on="last_update")

In [8]:
#transfor features
df["time"] = df["last_update"]
df["day"] = df["last_update"]

for i in range(df.shape[0]):
    df["time"].loc[i]= df.iloc[i]["last_update"].hour
    df["day"].loc[i] = df.iloc[i]["last_update"].dayofweek
    x= df.iloc[i]["last_update"].dayofweek

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [9]:
# implement weather data
#df["good_weather"] = np.logical_or(df["description"] == "Clouds", df["description"] == "Clear")
#df.good_weather.replace({True:1, False:0},inplace=True)
df

,number,bike_stands,available_bike_stands,available_bikes,last_update,id,description,temp,temp_max,temp_min,feels_like,wind_speed,sun_rise,sun_set,pressure,humidity,icon,time,day
0,4,20,8,12,2021-03-23 10:50:04,803,Clouds,8.47,9.44,7.78,2.74,6.17,2021-03-23 06:20:05,2021-03-23 18:43:09,1017,71,None,10,1
1,4,20,8,12,2021-03-23 10:53:18,803,Clouds,8.47,9.44,7.78,2.74,6.17,2021-03-23 06:20:05,2021-03-23 18:43:09,1017,71,None,10,1
2,4,20,8,12,2021-03-23 10:53:31,803,Clouds,8.47,9.44,7.78,2.74,6.17,2021-03-23 06:20:05,2021-03-23 18:43:09,1017,71,None,10,1
3,4,20,8,12,2021-03-23 10:53:47,803,Clouds,8.47,9.44,7.78,2.74,6.17,2021-03-23 06:20:05,2021-03-23 18:43:09,1017,71,None,10,1
4,4,20,7,13,2021-03-23 10:58:48,300,Drizzle,8.91,9.44,8.00,3.81,5.66,2021-03-23 06:20:05,2021-03-23 18:43:09,1017,76,None,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5188,4,20,13,7,2021-04-10 12:00:53,803,Clouds,6.42,7.22,5.00,4.53,2.57,2021-04-10 05:36:49,2021-04-10 19:15:33,1017,60,04d,12,5
5189,4,20,13,7,2021-04-10 12:05:54,803,Clouds,6.42,7.22,5.00,4.53,2.57,2021-04-10 05:36:49,2021-04-10 19:15:33,1017,60,04d,12,5
5190,4,20,12,8,2021-04-10 12:10:54,803,Clouds,6.42,7.22,5.00,4.53,2.57,2021-04-10 05:36:49,2021-04-10 19:15:33,1017,60,04d,12,5
5191,4,20,12,8,2021-04-10 12:15:55,803,Clouds,6.42,7.22,5.00,4.53,2.57,2021-04-10 05:36:49,2021-04-10 19:15:33,1017,60,04d,12,5


In [10]:
# create test and train sets
X = df[["time","day","humidity","temp"]]
y = df.available_bikes

X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [11]:
#linear model
model = LinearRegression()
model.fit(X_train, y_train)

print(model.coef_)
print(model.intercept_)

[-0.06128482 -0.77256419 -0.02155576 -0.15335845]
13.261541048825102


In [12]:
#run test set
model_predicitons = model.predict(X_test)
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(model_predicitons,columns=["Predicted"],index=y_test.index)],axis=1)

In [13]:
#compute error metrics for test set
actual_vs_predicted["squared_error"] = (actual_vs_predicted["available_bikes"] - actual_vs_predicted["Predicted"])**2
rmse_test = ((actual_vs_predicted["squared_error"].sum())/actual_vs_predicted.shape[0])**0.5
mae_test = ((actual_vs_predicted["squared_error"])**0.5).sum()/actual_vs_predicted.shape[0]

print("RMSE",rmse_test)
print("MAE",mae_test)

RMSE 3.835031931776902
MAE 2.9815456756175625


In [14]:
#colate this for single function
def get_model(n):
    """returns a model with error scores using the test set"""
    #get infor from db
    availability = '''SELECT *
    FROM availability
    WHERE number = {}
    '''.format(n)

    weather = '''SELECT *
    FROM weather
    '''

    select = """SELECT s.number
    FROM stations s"""
    
    #convert db tables to pandas dataframes
    dfs = pd.read_sql_query(select, engine)
    dfa = pd.read_sql_query(availability, engine)
    dfw = pd.read_sql_query(weather, engine)
    
    #merge weather and occupancy tables together
    df = pd.merge_asof(dfa, dfw, left_on="last_update", right_on="last_update")
    
    #transform existing features
    df["time"] = df["last_update"]
    df["day"] = df["last_update"]

    for i in range(df.shape[0]):
        df["time"].loc[i]= df.iloc[i]["last_update"].hour
        df["day"].loc[i] = df.iloc[i]["last_update"].dayofweek
    
    # create test and train sets
    X = df[["time","day","humidity","temp"]]
    y = df.available_bikes

    X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=0)
    
    #linear model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    #run test set
    model_predicitons = model.predict(X_test)
    actual_vs_predicted = pd.concat([y_test, pd.DataFrame(model_predicitons,columns=["Predicted"],index=y_test.index)],axis=1)
    
    actual_vs_predicted["squared_error"] = (actual_vs_predicted["available_bikes"] - actual_vs_predicted["Predicted"])**2
    rmse_test = ((actual_vs_predicted["squared_error"].sum())/actual_vs_predicted.shape[0])**0.5
    mae_test = ((actual_vs_predicted["squared_error"])**0.5).sum()/actual_vs_predicted.shape[0]
    
    #get model using full dataset
    model = LinearRegression()
    model.fit(X, y)
    results = [model,{"rmse":rmse_test, "mae":mae_test}]
    
    return results
                

In [15]:
stations = dfs.number.tolist()
stations[:3]

[2, 3, 4]

In [16]:
models = {}
for s in stations[:1]:
    models[s] = get_model(s)
    print("Station number :",s," Complete")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Station number : 2  Complete


In [17]:
models

{2: [LinearRegression(),
  {'rmse': 3.981643845140501, 'mae': 3.4258931371838224}]}

In [18]:
import pickle

In [19]:
#pickle
with open('models.pkl', 'wb') as handle:
    pickle.dump(models, handle, pickle.HIGHEST_PROTOCOL)

In [20]:
models = pickle.load(open('models.pkl','rb'))

In [29]:
def model(number, time, date):
    #import model from pickle file
    pickle_in = open("models.pkl", "rb")
    models = pickle.load(pickle_in)
    model = models[number][0]
    
    #compute bikes available at this time
    dic = {'day': 5, 'humid': 61, 'temp': 9.53}
    result = model.predict([np.array([time, dic['day'], dic['humid'], dic['temp']])])
    result = result[0]

    #ensures prediction cannot be negitive
    if result <= 0:
        result = 0
        
    return str(int(result))

In [30]:
model(2, 16, "2021-04-16")

'4'

In [15]:
#trains all models, and returns error scores
#WARNING. THIS FUNCTION WILL TAKE A LONG TIME AS YOU ARE RUNNING ALL CODE FEATURED IN THIS NOTEBOOK FOR
#EVERY MODE IN THE DATASET. However long it took you to run all code thus far, this code will repeat those task
#for over 100 stations instead of just 1.
models = {}
for s in stations:
    models[s] = get_model(s)
    print("Station number :",s," Complete")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Station number : 2  Complete


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Station number : 3  Complete


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


...

Station number : 115  Complete


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Station number : 116  Complete


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Station number : 117  Complete
